In [ ]:
# This imports the pandas library for data manipulation and analysis
import pandas as pd

movies = pd.read_csv("movies.csv") # This reads the CSV file named "tags.csv" and stores the data in the variable 'tags'
ratings = pd.read_csv("ratings.csv") # This reads the 'ratings.csv' file into a pandas DataFrame and assign it to the variable 'ratings'
tags = pd.read_csv("tags.csv") # This reads the 'tags.csv' file into a pandas DataFrame and assign it to the variable 'tags'

In [11]:
movies.shape()
movies.size()
movies.head()
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      62423 non-null  int64 
 1   title        62423 non-null  object
 2   genres       62423 non-null  object
 3   clean_title  62423 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.9+ MB


In [ ]:
ratings.shape()
ratings.size()
ratings.head()
ratings.info()

In [ ]:
tags.shape()
tags.size()
tags.head()
tags.info()

In [7]:
 # This imports the re module for regular expression operations
import re

In [8]:
# This imports the re module for regular expression operations
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)  # This removes any characters that are not alphanumeric or spaces from the 'title' string
    return title # This returns the cleaned 'title' string


In [9]:
# This adds a new column called "clean_title" to the 'movies' DataFrame
# The values in this column are obtained by applying the 'clean_title' function to the values in the "title" column
movies["clean_title"] = movies["title"].apply(clean_title)

In [ ]:
# This imports the TfidfVectorizer class from the sklearn.feature_extraction.text module.
from sklearn.feature_extraction.text import TfidfVectorizer

# This initializes an instance of the TfidfVectorizer class, named "vectorizer".
# This means that the vectorizer will consider both single words(unigrams) and pairs of words (bigrams)
vectorizer = TfidfVectorizer(ngram_range=(1,2))

# This applies the fit-transform method of the vectorizer to the "clean_title" column of the movies dataframe. This will convert the text data into a matrix of TF-IDF features
# Each row of the matrix represents a movie title, and each column represents a unique word or bigram. The values in the matrix represent the TF-IDF scores for each word or bigram in each movie title
tfidf = vectorizer.fit_transform(movies["clean_title"])

In [ ]:
# Importing the necessary libraries

# This imports the cosine_similarity function from sklearn.metrics.pairwise module. This function calculates the cosine similarity between two sets of samples by computing the dot product between them and normalizing it
# This produces a similarity matrix where each element represents the cosine similarity between two samples
from sklearn.metrics.pairwise import cosine_similarity
#This imports the numpy library
import numpy as np

def search(title):
    title = clean_title(title) # This cleans the title before processing
    query_vec = vectorizer.transform([title]) # This transforms the cleaned title into a vector using the vectorizer
    similarity = cosine_similarity(query_vec, tfidf).flatten() # This calculates the cosine similarity between the query vector and the tfidf matrix
    indices = np.argpartition(similarity, -5)[-5:] # This finds the indices of the top 5 most similar movies
    results = movies.iloc[indices].iloc[::-1] # This retrieves the movie information for the top 5 similar movies
    return results # This returns the results
    

In [ ]:
# Importing the necessary modules for creating interactive widgets and displaying output in Jupyter Notebook.
import ipywidgets as widgets
from IPython.display import display

# This creates a text input widget named movie_input with an initial value of 'Jumanji'.
movie_input = widgets.Text(
    value='Jumanji',
    description='Movie Title:', # This adds a description label 'Movie Title:'
    disabled=False # This enables the widget for user input.
)
# This creates an output widget named movie_list that will be used to display the search results.
movie_list = widgets.Output()

# This defines a function on_type that will be called whenever the value of movie_input changes.
def on_type(data):
    with movie_list: # This checks if the movie_list widget contains any data 
        movie_list.clear_output() # This clears the output widget movie_list
        title = data["new"] # This retrieves the new value of movie_input
        if len(title) > 5: # This checks if the length of the title is greater than 5 characters
            display(search(title)) # This displays the search results

movie_input.observe(on_type, names='value') # This sets up an observer on the movie_input widget, so that whenever its value changes, the on_type function will be called.


display(movie_input, movie_list) # This displays the movie_input widget and movie_list widget in the Jupyter Notebook.

Text(value='Jumanji', description='Movie Title:')

Output()

In [ ]:
movie_id = 26764

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

In [ ]:
# This removes rows with null values in 'tag' column from the original DataFrame 'tags'
tags.dropna(subset=["tag"], inplace=True)
#This removes rows where the length of the "tag" is less than 4 characters
tags.drop(tags[tags["tag"].str.len() < 4].index, inplace=True)
# This creates a boolean mask indicating which rows have tags with words less than 4 characters
mask = tags["tag"].str.split().apply(lambda x: any(len(word) < 4 for word in x))
# This filters out rows where the mask is True (i.e., tags with words less than 4 characters)
tags = tags[~mask]
# This gets the shape of the 'tags' DataFrame
tags.shape


(956656, 4)

In [ ]:
# Calculate recommendation percentages based on ratings
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
similar_user_recs_ratings = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
similar_user_recs_ratings = similar_user_recs_ratings.value_counts() / len(similar_users)

similar_user_recs_ratings = similar_user_recs_ratings[similar_user_recs_ratings > 0.10]

all_users_ratings = ratings[(ratings["movieId"].isin(similar_user_recs_ratings.index)) & (ratings["rating"] > 4)]
all_user_recs_ratings = all_users_ratings["movieId"].value_counts() / len(all_users_ratings["userId"].unique())

# Calculate recommendation percentages based on tags
similar_user_recs_tags = tags[tags["userId"].isin(similar_users)]["movieId"]
similar_user_recs_tags = similar_user_recs_tags.value_counts() / len(similar_users)

similar_user_recs_tags = similar_user_recs_tags[similar_user_recs_tags > 0.01]

all_users_tags = tags[tags["movieId"].isin(similar_user_recs_tags.index)]
all_user_recs_tags = all_users_tags["movieId"].value_counts() / len(all_users_tags["userId"].unique())

# Combine the recommendation percentages from ratings and tags
rec_percentages = pd.concat([similar_user_recs_ratings, all_user_recs_ratings, similar_user_recs_tags, all_user_recs_tags], axis=1)
rec_percentages.columns = ["similar_ratings", "all_ratings", "similar_tags", "all_tags"]

#rec_percentages.dropna(how='any', inplace=True)

rec_percentages


similar_ratings  all_ratings  similar_tags  all_tags
1                  0.25     0.121707           NaN       NaN
2                  0.50     0.017233           NaN       NaN
6                  0.50     0.047630           NaN       NaN
9                  0.25     0.001428           NaN       NaN
10                 0.25     0.021556           NaN       NaN
...                 ...          ...           ...       ...
202159             0.25     0.000026           NaN       NaN
202934             0.25     0.000039           NaN       NaN
203208             0.25     0.000039           NaN       NaN
203519             0.50     0.000090           NaN       NaN
206527             0.25     0.000013           NaN       NaN

[1928 rows x 4 columns]

In [ ]:
def find_similar_movies(movie_id):
   # Calculate recommendation percentages based on ratings
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs_ratings = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs_ratings = similar_user_recs_ratings.value_counts() / len(similar_users)

    similar_user_recs_ratings = similar_user_recs_ratings[similar_user_recs_ratings > 0.10]

    all_users_ratings = ratings[(ratings["movieId"].isin(similar_user_recs_ratings.index)) & (ratings["rating"] > 4)]
    all_user_recs_ratings = all_users_ratings["movieId"].value_counts() / len(all_users_ratings["userId"].unique())

    # Calculate recommendation percentages based on tags
    similar_user_recs_tags = tags[tags["userId"].isin(similar_users)]["movieId"]
    similar_user_recs_tags = similar_user_recs_tags.value_counts() / len(similar_users)

    similar_user_recs_tags = similar_user_recs_tags[similar_user_recs_tags > 0.01]

    all_users_tags = tags[tags["movieId"].isin(similar_user_recs_tags.index)]
    all_user_recs_tags = all_users_tags["movieId"].value_counts() / len(all_users_tags["userId"].unique())

    # Combine the recommendation percentages from ratings and tags
    rec_percentages = pd.concat([similar_user_recs_ratings, all_user_recs_ratings, similar_user_recs_tags, all_user_recs_tags], axis=1)
    rec_percentages.columns = ["similar_ratings", "all_ratings", "similar_tags", "all_tags"]

    #rec_percentages.dropna(how='any', inplace=True)
    
    rec_percentages = rec_percentages.fillna(1)
    rec_percentages["score"] = (rec_percentages["similar_ratings"] / rec_percentages["all_ratings"]) * (rec_percentages["similar_tags"] / rec_percentages["all_tags"])
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [ ]:
print("Enter Your Movie Title and Get Recommendations of Similar Movies that I'm sure you would like")
import ipywidgets as widgets
from IPython.display import display

# This creates a text input widget for the movie title
movie_name_input = widgets.Text(
    value='Transformers',
    description='Movie Title:',
    disabled=False
)

#This creates an output widget to display the recommendations
recommendation_list = widgets.Output()

# This creates a function to handle the text input
def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title) # This performs a search based on the movie title
            movie_id = results.iloc[0]["movieId"] # This displays the similar movies based on the search results
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value') # This conncects the event handler to the text input widget

display(movie_name_input, recommendation_list)  # This displays the text input widget and the output widget

Enter Your Movie Title and Get Recommendations of Similar Movies that I'm sure you would like


Text(value='Transformers', description='Movie Title:')

Output()